In [30]:
import openslide, numpy as np
import cv2
from cv2 import filter2D
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.cluster import KMeans
import os
import bitarray
from bitarray import util as butil
from glob import glob 

os.environ['NVIDIA_VISIBLE_DEVICES'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import tensorflow as tf

In [25]:
class BoB:
    
    def __init__(self, barcodes):
        self.barcodes = [bitarray.bitarray(b.tolist()) for b in barcodes]
        
    def select_subset(self, n = 3):
        idx = np.arange(len(self.barcodes))
        np.random.shuffle(idx)
        idx = idx[:n]
        
        return BoB(barcodes=[self.barcodes[i] for i in idx])
    
    def distance(self, bob):

        total_dist = []
        for feat in self.barcodes:
            distances = [butil.count_xor(feat, b) for b in bob.barcodes]
            total_dist.append(np.min(distances))

        retval = np.median(total_dist)

        return retval

In [38]:
def preprocessing_fn(inp, sz=(1000, 1000)):

    out = tf.cast(inp, 'float') / 255.
    
    out = tf.cond(tf.equal(tf.shape(inp)[1], sz[0]), 
                lambda: out, lambda: tf.image.resize(out, sz))
    
    mean = tf.reshape((0.485, 0.456, 0.406), [1, 1, 1, 3])
    std = tf.reshape((0.229, 0.224, 0.225), [1, 1, 1, 3])
    
    out = out - mean
    out = out / std
    
    return out


def get_dn121_model():
    model = tf.keras.applications.densenet.DenseNet121(input_shape=(1000, 1000, 3),\
                                                       include_top=False,\
                                                       pooling='avg')
    
    seq_model = tf.keras.models.Sequential([tf.keras.layers.Lambda(preprocessing_fn,\
                                                   input_shape=(None, None, 3),\
                                                   dtype=tf.uint8)])
    seq_model.add(model)
    return seq_model

# https://github.com/FarhadZanjani/Histopathology-Stain-Color-Normalization/blob/master/ops.py
def RGB2HSD(X):
    eps = np.finfo(float).eps
    X[np.where(X==0.0)] = eps
    
    OD = -np.log(X / 1.0)
    D  = np.mean(OD,3)
    D[np.where(D==0.0)] = eps
    
    cx = OD[:,:,:,0] / (D) - 1.0
    cy = (OD[:,:,:,1]-OD[:,:,:,2]) / (np.sqrt(3.0)*D)
    
    D = np.expand_dims(D,3)
    cx = np.expand_dims(cx,3)
    cy = np.expand_dims(cy,3)
            
    X_HSD = np.concatenate((D,cx,cy),3)
    return X_HSD


def clean_thumbnail(thumbnail):
    thumbnail_arr = np.asarray(thumbnail)
    
    # writable thumbnail
    wthumbnail = np.zeros_like(thumbnail_arr)
    wthumbnail[:, :, :] = thumbnail_arr[:, :, :]

    # Remove pen marking here
    # We are skipping this
    
    # This  section sets regoins with white spectrum as the backgroud regoin
    thumbnail_std = np.std(wthumbnail, axis=2)
    wthumbnail[thumbnail_std<5] = (np.ones((1,3), dtype="uint8")*255)
    thumbnail_HSD = RGB2HSD( np.array([wthumbnail.astype('float32')/255.]) )[0]
    kernel = np.ones((30,30),np.float32)/900
    thumbnail_HSD_mean = cv2.filter2D(thumbnail_HSD[:,:,2],-1,kernel)
    wthumbnail[thumbnail_HSD_mean<0.05] = (np.ones((1,3),dtype="uint8")*255)
    return wthumbnail

def process_slide(slide_fn):
    slide = openslide.open_slide(slide_fn)
    objective_power = int(slide.properties['openslide.objective-power'])
    w, h = slide.dimensions

    # at 20x its 1000x1000
    patch_size = (objective_power/20.)*1000
    
    ##############
    ## Patch it ##
    ##############
    mask_hratio = (tissue_mask.shape[0]/h)*patch_size
    mask_wratio = (tissue_mask.shape[1]/w)*patch_size

    # iterating over patches
    patches = []

    for i, hi in enumerate(range(0, h, int(patch_size) )):

        _patches = []
        for j, wi in enumerate(range(0, w, int(patch_size) )):

            # check if patch contains 70% tissue area
            mi = int(i*mask_hratio)
            mj = int(j*mask_wratio)

            patch_mask = tissue_mask[mi:mi+int(mask_hratio), mj:mj+int(mask_wratio)]

            tissue_coverage = np.count_nonzero(patch_mask)/patch_mask.size

            _patches.append({'loc': [i, j], 'wsi_loc': [int(hi), int(wi)], 'tissue_coverage': tissue_coverage})

        patches.append(_patches)
        
    # for patch to be considered it should have this much tissue area
    tissue_threshold = 0.7

    flat_patches = np.ravel(patches)
    for patch in tqdm(flat_patches):

        # ignore patches with less tissue coverage
        if patch['tissue_coverage'] < tissue_threshold:
            continue

        # this loc is at the objective power
        h, w = patch['wsi_loc']

        # we will go obe level lower, i.e. (objective power / 4)
        # we still need patches at 5x of size 250x250
        # this logic can be modified and may not work properly for images of lower objective power < 20 or greater than 40
        patch_size_5x = int(((objective_power / 4)/5)*250.)

        patch_region = slide.read_region((w, h), 1, (patch_size_5x, patch_size_5x)).convert('RGB')

        if patch_region.size[0] != 250:
            patch_region = patch_region.resize((250, 250))

        histogram = (np.array(patch_region)/255.).reshape((250*250, 3)).mean(axis=0)
        patch['rgb_histogram'] = histogram    

    selected_patches_flags = [patch['tissue_coverage'] >= tissue_threshold for patch in flat_patches]
    selected_patches = flat_patches[selected_patches_flags]

    kmeans_clusters = 9
    kmeans = KMeans(n_clusters = kmeans_clusters)
    features = np.array([entry['rgb_histogram'] for entry in selected_patches])

    kmeans.fit(features)
    
    patch_clusters = np.zeros(np.array(patches).shape+(3,))

    for patch, label in zip(selected_patches, kmeans.labels_):
        patch_clusters[patch['loc'][0], patch['loc'][1], :] = cmap(label)[:3]
        patch['cluster_lbl'] = label

    # Another hyperparameter of Yottixel
    # Yottixel has been tested with 5, 10, and 15 with 15 performing most optimally
    percentage_selected = 15

    mosaic = []

    for i in range(kmeans_clusters):
        cluster_patches = selected_patches[kmeans.labels_ == i]
        n_selected = max(1, int(len(cluster_patches)*percentage_selected/100.))

        km = KMeans(n_clusters=n_selected)
        loc_features = [patch['wsi_loc'] for patch in cluster_patches]
        ds = km.fit_transform(loc_features)

        c_selected_idx = []
        for idx in range(n_selected):
            sorted_idx = np.argsort(ds[:, idx])

            for sidx in sorted_idx:
                if sidx not in c_selected_idx:
                    c_selected_idx.append(sidx)
                    mosaic.append(cluster_patches[sidx])
                    break

    patch_clusters = np.zeros(np.array(patches).shape+(3,))

    for patch in selected_patches:
        patch_clusters[patch['loc'][0], patch['loc'][1], :] = np.array(cmap(patch['cluster_lbl'])[:3])*0.6
    for patch in mosaic:
        patch_clusters[patch['loc'][0], patch['loc'][1], :] = cmap(patch['cluster_lbl'])[:3]

    model = get_dn121_model()

    patch_queue = []
    feature_queue = []
    batch_size = 20

    for patch in tqdm(mosaic):

        # this loc is at the objective power
        h, w = patch['wsi_loc']

        patch_size_20x = int((objective_power/20.)*1000)
        patch_region = slide.read_region((w, h), 0, (patch_size_20x, patch_size_20x)).convert('RGB')

        patch_queue.append(np.array(patch_region))
        if len(patch_queue) == batch_size:
            feature_queue.extend(model.predict( np.array(patch_queue) ))
            patch_queue = []

    if len(patch_queue) != 0:
        padded_arr = np.zeros((batch_size, patch_size_20x, patch_size_20x, 3))
        padded_arr[:len(patch_queue), :, :, :] = np.array(patch_queue)
        feature_queue.extend(model.predict( padded_arr )[:len(patch_queue)])

    bob_raw = (np.diff(np.array(feature_queue), axis=1) < 0)*1
    bob = BoB(bob_raw)
    return bob

In [39]:
slide_path = '/mnt/storage/COMET/RAW/*.svs'
slides = glob(slide_path)

In [40]:
bobs = []
for slide in tqdm(slides[:5]):
    bobs.append(process_slide(slide))

100%|██████████| 2322/2322 [00:27<00:00, 83.02it/s] 

100%|██████████| 72/72 [00:00<00:00, 116.76it/s]

 56%|█████▌    | 943/1683 [00:14<00:22, 33.47it/s]


100%|██████████| 1683/1683 [00:20<00:00, 84.05it/s] 

100%|██████████| 1638/1638 [00:18<00:00, 86.30it/s] 

 37%|███▋      | 1150/3105 [00:10<00:21, 92.05it/s]


100%|██████████| 3105/3105 [00:37<00:00, 83.65it/s] 

100%|██████████| 5/5 [09:16<00:00, 111.21s/it]
